# [1] 라이브러리 설치

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 7.5 MB/s 
     |████████████████████████████████| 448 kB 53.4 MB/s 
     |████████████████████████████████| 86 kB 8.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install kss

     |████████████████████████████████| 65 kB 3.8 MB/s 


In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from konlpy.utils import pprint
from hanspell import spell_checker
import io
import pandas as pd
import kss
import re

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
!pip install Selenium
!apt-get update 
!apt install chromium-chromedriver

# [2] 파일 import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import io
import pandas as pd
import kss
import re

In [ ]:
# raw 데이터 셋 불러오기
elxcle_file = '/content/drive/MyDrive/opinionlive/data/rawdata(command).xlsx' ##경로확인
file_elxcle = pd.read_excel(elxcle_file)

In [ ]:
file_elxcle

,카테고리,구문태킹,Unnamed: 2,Unnamed: 3,원본댓글,Unnamed: 5
0,명령,승인해라,"('승인', 'Noun'), ('해라', 'Verb'), ('.', 'Punctua...",Noun + Verb,이버맥틴 승인해라.,NaN
1,명령,선점하지마라,"('선점', 'Noun'), ('하지마라', 'Verb')]",Noun + Verb,낮은게아니라 없어서 못 맞는거다 미국이나 유럽에서 부스터샷한다고 선점하지마라,NaN
2,명령,감면해줘라,"('감면', 'Verb'), ('해줘라', 'Verb'), ('.', 'Punctu...",Verb+ Verb,죽어나가는 소상공인 이자나 좀 감면해줘라.,NaN
3,명령,명심들 하세요!,"('명심', 'Noun'), ('들', 'Suffix'), ('하세요', 'Verb...",Noun + Suffix + Verb,공산당이 하는 말은 백해무익한 시뻘건 거짓말! 명심들 하세요!,NaN
4,명령,그런거 하지 마세요,"('그런거', 'Adjective'), ('하지', 'Verb'), ('마세요', ...",Adjective + Verb + Verb,올말까지 백신 그런거 하지마세요.,NaN
...,...,...,...,...,...,...
121,주장,죽어야 한다,"('죽어야', 'Verb'), ('한다', 'Verb')",Verb + Verb,박수현의 말이 사실이라면 서욱장관 이하 국방부 관계자는 혀깨물고 죽어야 한다.,NaN
122,주장,데모해야된다,"('해야', 'Verb'), ('된다', 'Verb')",Verb + Verb,미국도 영국처럼 데모해야된다.,NaN
123,주장,해야한다,"('해야', 'Verb'), ('한다', 'Verb')",Verb + Verb,만일 수송기 급파가 국방부나 합참의 메뉴얼에 있는 사항이라면 문재인은맨땅에 해딩하고...,NaN
124,주장,바보인 것임,"('바보', 'Noun'), ('인', 'Josa'), ('것임', 'Noun')",Noun + Josa + Noun,"문파, 국짐당, 제약회사 댓글알바들에게 선동되어 백신 맞는 놈들이 바보인 것임...",NaN


## 파일 정리

In [ ]:
# 댓글만 뽑기
command = file_elxcle['command']
command = command.dropna(axis=0)

In [ ]:
command

0                                             이버맥틴 승인해라.
1             낮은게아니라 없어서 못 맞는거다 미국이나 유럽에서 부스터샷한다고 선점하지마라
2                                죽어나가는 소상공인 이자나 좀 감면해줘라.
3                     공산당이 하는 말은 백해무익한 시뻘건 거짓말! 명심들 하세요!
4                                     올말까지 백신 그런거 하지마세요.
                             ...                        
121          박수현의 말이 사실이라면 서욱장관 이하 국방부 관계자는 혀깨물고 죽어야 한다.
122                                     미국도 영국처럼 데모해야된다.
123    만일 수송기 급파가 국방부나 합참의 메뉴얼에 있는 사항이라면 문재인은맨땅에 해딩하고...
124       문파, 국짐당, 제약회사 댓글알바들에게 선동되어 백신 맞는 놈들이 바보인 것임...
125                                              백신은 사기임
Name: 원본댓글, Length: 126, dtype: object

# [3] 맞춤법 검사

## 부산대 맞춤법 -> 한셀 

In [ ]:
from selenium import webdriver
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') 
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [ ]:
def spell_checking(sentence): 
  time.sleep(0.5) 
  wd.get('http://speller.cs.pusan.ac.kr/') 
  try: 
    wd.find_element_by_xpath('//*[@id="text1"]').send_keys(sentence) 
    wd.find_element_by_xpath('//*[@id="btnCheck"]').click() 
    time.sleep(2) 
    entity_num = 0 
    while True: 
      try:
         wd.find_element_by_xpath('//*[@id="tdReplaceWord_'+str(entity_num)+'"]/ul/li/a').click() 
         entity_num += 1 
      except: 
        break 
    texts = wd.find_element_by_xpath('//*[@id="tdCorrection1stBox"]').text 
    return texts 
  except: 
    return sentence

In [ ]:
pusan = []
a = 0
#부산대 맞춤법
for i in command : 
  a = a+1
  print(i,"\n--",spell_checking(i))
  pu = spell_checking(i)
  pusan.append(pu)

In [ ]:
 swu = []
#한셀
  for a in pusan:
    spelled_sent = spell_checker.check(str(a))
    hanspell_sent = spelled_sent.checked
    
    swu.append(hanspell_sent)

# [4] 문장단위 토큰화

In [ ]:
  #문장단위 토큰화
  sentences = []
  a = 0
  for i in command:
    a = a+1
    print(a,":",i)
    sent = kss.split_sentences(i)
    sentences = sentences + sent
    print(a,":",sent)
    if a == 50 :
      break
  print(len(sentences))

1 : 이버맥틴 승인해라.
1 : ['이버맥틴 승인해라.']
2 : 낮은게아니라 없어서 못 맞는거다 미국이나 유럽에서 부스터샷한다고 선점하지마라
2 : ['낮은게아니라 없어서 못 맞는거다 미국이나 유럽에서 부스터샷한다고 선점하지마라']
3 : 죽어나가는 소상공인 이자나 좀 감면해줘라.
3 : ['죽어나가는 소상공인 이자나 좀 감면해줘라.']
4 : 공산당이 하는 말은 백해무익한 시뻘건 거짓말! 명심들 하세요!
4 : ['공산당이 하는 말은 백해무익한 시뻘건 거짓말! 명심들 하세요!']
5 : 올말까지 백신 그런거 하지마세요.
5 : ['올말까지 백신 그런거 하지마세요.']
6 : 어짜피 망한 나라인데 재인이형 맘대로 하세요.
6 : ['어짜피 망한 나라인데 재인이형 맘대로 하세요.']
7 : 정부가 똥먹으라고하면 똥먹을거야?ㅋㅋㅋㅋ 정부말잘듣고 사세요
7 : ['정부가 똥먹으라고하면 똥먹을거야?ㅋㅋㅋㅋ 정부말잘듣고 사세요']
8 : 이나라를 누가이렇게만들었나 끝나면책임물어야지 전부개판이니
8 : ['이나라를 누가이렇게만들었나 끝나면책임물어야지 전부개판이니']
9 : 세금으로 모두 환수해라
9 : ['세금으로 모두 환수해라']
10 : 한강구역을 통제해서 물 오염되지 못하게 특공대 투입해서 막고 번개탄 판매 중지해라!!
10 : ['한강구역을 통제해서 물 오염되지 못하게 특공대 투입해서 막고 번개탄 판매 중지해라!!']
11 : 도박하면 죽인다는 걸 인식시켜 다신 2030 도박 못하도록 법개정도 하고 하면 10년이하의 징역 때려라.
11 : ['도박하면 죽인다는 걸 인식시켜 다신 2030 도박 못하도록 법개정도 하고 하면 10년이하의 징역 때려라.']
12 : 손가락 짜르는 법안도 통과시켜라
12 : ['손가락 짜르는 법안도 통과시켜라']
13 : 부가가치는 그대로인데 월급통장 숫자만 더 많이 찍히게 만들면 부자가 된다는 초등학생 논리.. 그럴거면 그냥 조폐국에서 하루 1000조씩 찍어서 헬기로 전국에 살포하십시오.
13 : ['부가가치는 그대로인데 월급통장 숫

In [ ]:
result_excel=[]

# [5] okt형태소 분석

In [ ]:
for line in sentences:
  okt_command = okt.pos(line)
  print(line, ":",okt_command)
  result_excel.append(okt_command)

이버맥틴 승인해라. : [('이버맥틴', 'Noun'), ('승인', 'Noun'), ('해라', 'Verb'), ('.', 'Punctuation')]
낮은게아니라 없어서 못 맞는거다 미국이나 유럽에서 부스터샷한다고 선점하지마라 : [('낮은게', 'Adjective'), ('아니라', 'Adjective'), ('없어서', 'Adjective'), ('못', 'Noun'), ('맞는거다', 'Verb'), ('미국', 'Noun'), ('이나', 'Josa'), ('유럽', 'Noun'), ('에서', 'Josa'), ('부스터', 'Noun'), ('샷', 'Noun'), ('한다고', 'Verb'), ('선점', 'Noun'), ('하지마라', 'Verb')]
죽어나가는 소상공인 이자나 좀 감면해줘라. : [('죽어', 'Verb'), ('나가는', 'Verb'), ('소상', 'Noun'), ('공인', 'Noun'), ('이자나', 'Josa'), ('좀', 'Noun'), ('감면', 'Verb'), ('해줘라', 'Verb'), ('.', 'Punctuation')]
공산당이 하는 말은 백해무익한 시뻘건 거짓말! 명심들 하세요! : [('공산당', 'Noun'), ('이', 'Josa'), ('하는', 'Verb'), ('말', 'Noun'), ('은', 'Josa'), ('백해무익한', 'Adjective'), ('시뻘건', 'Adjective'), ('거짓말', 'Noun'), ('!', 'Punctuation'), ('명심', 'Noun'), ('들', 'Suffix'), ('하세요', 'Verb'), ('!', 'Punctuation')]
올말까지 백신 그런거 하지마세요. : [('올', 'Modifier'), ('말', 'Noun'), ('까지', 'Josa'), ('백신', 'Noun'), ('그런거', 'Adjective'), ('하지', 'Verb'), ('마세요', 'Verb'), ('.', 'Punctuation')]
어짜피 망

In [ ]:
# results_df = pd.DataFrame(result_excel)
# results_df.columns = ['원문','토큰화']
# results_df.to_excel('/content/drive/MyDrive/OpinionLIVE/크롤링_토큰화결과.xlsx',encoding='utf-8-sig')

# [6] 구문 분석

## grammar

In [ ]:
from nltk.chunk.regexp import RegexpParser

In [ ]:
grammar = RegexpParser("""
  AA: {<Adverb><Adjective>}
  BB: {<Adverb><Verb>}
  VV: {<Verb>+<Verb>+}
  AD: {<Adjective><Adjective>} #없던게 아닐까
  NP4: {<Noun><Verb>*<Josa>*<Adjective>+}
  NP0: {<Noun><Verb>}
  VP4: {<VerbPrefix><Verb>}
  NP1: {<Noun>?<Noun>?<Noun>?<Suffix>*<Verb>+}
  NP2: {<Noun>+<Josa>+}
  NP3: {<Noun>+<Suffix>+<Josa>+}
  NP5: {<Noun>?<Noun>?<Noun>?<Noun>?}
  VP1: {<Adjective>+<Verb>+}
  VP2: {<Verb>+<Adjective>+}
  VP3: {<Noun>+<Suffix>*<Adjective>+}
""")

## 구문으로 나누기

In [ ]:
inu = []
b = 0
parse_list = []

for i in result_excel:
  b = b+1
  parseTree = grammar.parse(i)
  subParse_list =[]
  for item in parseTree:
    eStr =''
    if (str(type(item[0])) == "<class 'tuple'>"):
      for j in item:
        eStr+=j[0]
        eStr+=' '
    elif (str(type(item[0])) == "<class 'str'>"):
      eStr+=item[0]
    subParse_list.append(eStr)
  oneS_subParse = [subParse_list]
  print('>>>>',oneS_subParse)
  parse_list.append(oneS_subParse)
  print("----------------------------")
  inu.append(parseTree)

>>>> [['내 가 ', '하고자 ', '해 ']]
----------------------------


## 엑셀 파일로 저장하기

In [ ]:
results_df34 = pd.DataFrame(parse_list)
results_df34.columns = ['parse']
results_df34.to_excel('/content/drive/MyDrive/opinionlive/data/parse.xlsx',encoding='utf-8-sig')

# ** okt 사용자 사전

## konlpy 라이브러리 설치

In [ ]:
import os

# KoNLPy 라이브러리 설치
!pip install konlpy

# jdk, JPype1-py3 설치
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

# automake 설치
os.chdir('/tmp')
!curl -LO http://ftpmirror.gnu.org/automake/automake-1.11.tar.gz
!tar -zxvf automake-1.11.tar.gz
os.chdir('/tmp/automake-1.11')
!./configure
!make
!make install

# automake 설치 오류 시
os.chdir('/tmp/') 
!wget -O m4-1.4.9.tar.gz http://ftp.gnu.org/gnu/m4/m4-1.4.9.tar.gz
!tar -zvxf m4-1.4.9.tar.gz
os.chdir('/tmp/m4-1.4.9')
!./configure
!make
!make install
os.chdir('/tmp')
!curl -OL http://ftpmirror.gnu.org/autoconf/autoconf-2.69.tar.gz
!tar xzf autoconf-2.69.tar.gz
os.chdir('/tmp/autoconf-2.69')
!./configure --prefix=/usr/local
!make
!make install
!export PATH=/usr/local/bin

ERROR: Operation cancelled by user
     |████████████████████████████████| 88 kB 5.2 MB/s 
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp37-cp37m-linux_x86_64.whl size=2682601 sha256=4dbaf538045e7daedc69a649cc4391e3d58d92dde5ea206fa59da368cf73f21c
  Stored in directory: /root/.cache/pip/wheels/e7/d1/09/f55dca0203b0691945bdf0f63d486a0b4d4e5ec4bd78a2502e
Successfully built JPype1-py3
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1339k  100 1339k    0     0  1116k      0  0:00:01  0:00:01 --:--:-- 2126k
automake-1.11/
automake-1.11/THANKS
automake-1.11/ChangeLog.03
automake-1.11/automake.in
automake-1.11/ChangeLog.96
automake-1.11/ChangeLog.98
automake-1.11/m4/
automake-1.11/m4/gcj.m4
automake-1.11/m4/missing.m4
automake-1.11/m4/runlog.m4
automake-1.11/m4/lex.m4
automake-1.11/m4/va

## 1) aaa폴더 추가

konlpy의 open-korean-text-2.1.0.jar의 압축푼파일을 새로운 aaa폴더안에 추가하기 위해 aaa폴더 추가함



/usr/local/lib/python3.6/dist-packages/konlpy

In [ ]:
import os
os.chdir('/usr/local/lib/python3.7/dist-packages/konlpy/java')
os.getcwd() 
os.makedirs('./aaa')

FileNotFoundError: ignored

## 2) 경로 확인 

현재 java 폴더여야함

In [ ]:
cd '/usr/local/lib/python3.7/dist-packages/konlpy/java'

In [ ]:
os.getcwd()

## 3) open-korean-text-2.1.0.jar폴더 압축 풀기

In [ ]:
!jar xvf ./open-korean-text-2.1.0.jar

## 4) org, META-INF 파일을 aaa폴더 안에 **넣기**

In [ ]:
cd '/usr/local/lib/python3.7/dist-packages/konlpy/java'

In [ ]:
mv org META-INF /usr/local/lib/python3.7/dist-packages/konlpy/java/aaa

## 5) 파일열기

noun이든지 verb든지 해당되는 품사 파일 열기

In [ ]:
#noun
# with open(f"/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/nouns.txt") as f:
#     data = f.read()

#verb
with open(f"/usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/org/openkoreantext/processor/util/verb/verb.txt") as f:
    data = f.read()

## 6) 추가할 단어 넣고 저장

In [ ]:
### 추가할 단어
data += '하려고\n하소서\n'

###파일 새롭게 저장
#noun
# with open("/usr/local/lib/python3.7/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/nouns.txt", 'w') as f:
#     f.write(data)
#verb
with open("/usr/local/lib/python3.7/dist-packages/konlpy/java/aaa/org/openkoreantext/processor/util/verb/verb.txt", 'w') as f:
    f.write(data)

## 7) 현재 path aaa폴더로 가기

In [ ]:
cd '/usr/local/lib/python3.7/dist-packages/konlpy/java/aaa'

In [ ]:
os.getcwd() 

## 8) 파일 압축하기

In [ ]:
!jar cvf open2-korean-text-2.1.0.jar .

## 9) 두개 파일 삭제하고 aaa폴더도 삭제하기

이제 압축푼 파일은 필요가 없으니까

In [ ]:
rm -rf 'META-INF'

In [ ]:
rm -rf 'org'

In [ ]:
# 압축한 파일 옮기기
mv open2-korean-text-2.1.0.jar /usr/local/lib/python3.7/dist-packages/konlpy/java

In [ ]:
cd ..

In [ ]:
rm -rf 'aaa'

## 10) 파일 정리

open-korean-text-2.1.0.ja삭제하고
open2-korean-text-2.1.0.ja 이름바꾸기

## 11) 런타임 다시 시작하고 위에 mecab-ko다시 실행

#### okt import하고 다시 실행하기 바꾼거 확인해보기

In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
print(okt.pos('내가 하고자 해'))